In [30]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

In [16]:
def get_data(url):
    hdr = {'User-Agent': 'Mozilla/5.0'}
    html = requests.get(url, headers=hdr).content
    soup=BeautifulSoup(html)
    name=soup.find('span', {'itemprop':'name'}).text
    df=pd.DataFrame({'name':name}, index=[0])
    for i in range(7,10):
        df_aux=pd.read_html(html)[7].T
        df_aux.columns=df_aux.iloc[0,:]
        df_aux=df_aux.loc[1,:]
        df_aux=pd.DataFrame(df_aux).T
        df_aux=df_aux.reset_index(drop=True)
        df=pd.concat([df,df_aux], axis=1)
    return df

In [26]:
 def get_players_link():
    url = 'https://fmdataba.com/20/l/191/bundesliga/best-players/'
    hdr = {'User-Agent': 'Mozilla/5.0'}
    html = requests.get(url, headers=hdr).content
    soup=BeautifulSoup(html)
    teams=soup.find_all('table', {'class':'table table-striped'})[1].find_all('td')
    url_teams=[item.find('a')['href'] for item in teams]
    url_list = []
    for team in url_teams:
        html = requests.get(team, headers=hdr).content
        soup=BeautifulSoup(html)
        players=soup.find('table', {'id':'tablo61'}).find_all('td', {'style':'font-size:12px;'})
        url_players=[item.find('a')['href'] for item in players]
        url_list.append(url_players)
    url_list=[item for x in url_list for item in x]
    return url_list

In [10]:
df_result = pd.DataFrame()
for url in tqdm(get_players_link()):
    try:
        df_aux = get_data(url)
        df_result = pd.concat([df_result, df_aux])
    except:
        pass

100%|██████████| 486/486 [19:30<00:00,  2.41s/it]
